In [1]:
# Imports and setting adjustments
import holopy as hp, numpy as np
import holopy.core as hpc
import holopy.core.io as hpci
from holopy.core.process import bg_correct, subimage, normalize
from holopy.scattering import calc_holo, Sphere, Mie
from holopy.inference import prior, ExactModel, TemperedStrategy, AlphaModel
%matplotlib qt
#path = r'C:\Users\kalis\Desktop\UMNJuniorYear\MXP2\Video Microscopy\dataverse_files\spheroid'

# Information on specific setup
# illumination wavelength -> .630 (633 nm)
# spacing and wavelength must be given in the same units (no required units, just consistency)

In [ ]:
# Define parameters of camera and medium
# https://aip.scitation.org/doi/10.1063/5.0015976
p_size = 0.05 #pixel size
noise = 0.0318 #average noise
med_n = 1.33 #medium refractive index
illum_wavelength = 0.660 #wavelength in vacuum
pol = (1,0) #polarization vector
print('Block 2 works!')

In [ ]:
# Takes the file path for hologram and allows for visualization
raw_holo = hp.load_image(path+r'\raw0\im0000.tif', spacing = p_size,channel = 0)
#hp.show(raw_holo)

# Creates hologram of (1) background intensity image and corrects raw_holo
#bkgd = hp.load_image(path+r'\bg0000.tif', spacing = p_size, channel = 0)
#holo = bg_correct(raw_holo, bkgd)
#hp.show(holo)

# Creates hologram of multiple background intensity images and corrects raw_holo
bkgd = hpci.load_average([path+r'\bg0000.tif',path+r'\bg0001.tif',path+r'\bg0002.tif',
                          path+r'\bg0003.tif'],refimg = raw_holo)
holo = bg_correct(raw_holo, bkgd)
hp.show(holo)

# Apply dark field images to avoid effects of stray light
#df = hpci.load_average([path+r'\dark0000.tif',path+r'\dark0001.tif',path+r'\dark0002.tif',
#                       path+r'\dark0003.tif'],refimg = raw_holo)
#holo = bg_correct(raw_holo, bkgd, df)
#hp.show(holo)

# Update information on hologram to include information on apparatus
holo = hpc.update_metadata(holo, med_n, illum_wavelength, pol)

# Save processed holograms
#hp.save_image('processed_holo.tif', holo)

holo = subimage(holo,[623,561],100)
holo = normalize(holo)
#hp.show(holo)
#hp.save_image('processed_holo.tif', holo)

In [ ]:
# Defines 3D volume from 2D image
zstack = np.linspace(0,20,20) # defines space to project data onto
rec_vol = hp.propagate(holo,zstack) #propagates image onto zstack

#Shows image - note that it only shows image magnitude
hp.show(rec_vol)

#Shows the imaginary part of the image
#hp.show(rec_vol.imag)

#Shows the phase angle corresponding to the image
#Phase is shown with different contrasts because of different locations in the phase period
#hp.show(np.angle(rec_vol))

#Corrects for differing contrasts in angle plot by going in intervals of phase period
#med_wavelen = holo.illum_wavelen / holo.medium_index
#rec_vol = hp.propagate(holo, zstack*med_wavelen)
#hp.show(rec_vol.imag)

In [ ]:
#define sphere for fit work
parameter_n = prior.Gaussian(1.60, 0.02)
parameter_r = prior.BoundedGaussian(0.65, 0.05, 0, np.inf)
parameter_center = prior.make_center_priors(holo)
#parameter_n = 1.59
#parameter_r = 0.05 #0.9
#parameter_center = (30, 29, 4)
#detector = hp.detector_grid(shape=100, spacing=0.1)



parameter_sphere = Sphere(parameter_n, parameter_r, parameter_center)
init_holo = calc_holo(rec_vol,parameter_sphere, med_n, illum_wavelength, pol, theory='auto')
hp.show(init_holo)

forward_model = AlphaModel(parameter_sphere, noise)
strategy = TemperedStrategy()
result = strategy.sample(forward_model, holo)

fit_sphere = parameter_sphere.from_parameters(result.values())

print('all done!')

In [ ]:
#Defines wanted parameters and performs fit
fit_result = hp.fit(holo, fit_sphere)
print('FINALLY!!')

# Output statements for resulting fit
print('Fit parameters: ')
print(fit_result.parameters)
print('\n')

print('Best Fit Sphere: ')
print(fit_result.scatterer)
print('\n')

hp.show(fit_result.hologram)

In [2]:
path = r'C:\Users\kalis\Downloads'

p_size = 0.5 #pixel size
noise = 0.3 #average noise
med_n = 1.33 #medium refractive index
illum_wavelength = 0.633 #wavelength in vacuum
pol = (1,0) #polarization vector

#Define raw image
raw_holo = hp.load_image(path+r'\stuckparticles2.tif', spacing = p_size, channel = 0)
#hp.show(raw_holo)

#Background Correction
bkgd = hpci.load_average([path+r'\bkgd1.tif',path+r'\bkgd2.tif',path+r'\bkgd3.tif',
                          path+r'\bkgd4.tif'],refimg = raw_holo, channel = 0)
holo = bg_correct(raw_holo, bkgd)
#hp.show(holo)

#Update information stored with hologram and normalize
holo = hpc.update_metadata(holo, med_n, illum_wavelength, pol)
holo = normalize(holo)

#Project on 3D space
zstack = np.linspace(0,20,20) # defines space to project data onto
rec_vol = hp.propagate(holo,zstack) #propagates image onto zstack
hp.show(rec_vol)

#Define fit sphere
parameter_n = 1.33
parameter_r = 1.5
parameter_center = (90, 60, 0.15)
parameter_sphere = Sphere(parameter_n, parameter_r, parameter_center)

#Visualize Estimate
#init_holo = calc_holo(rec_vol,parameter_sphere, med_n, illum_wavelength, pol, theory='auto')
#hp.show(init_holo)

#Perform Fit and display results
fit_result = hp.fit(holo, parameter_sphere)
print('Best Fit Sphere: ')
print(fit_result.scatterer)
print('\n')
hp.show(fit_result.hologram)

C:\Users\kalis\anaconda3\lib\site-packages\holopy\core\io\vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


Best Fit Sphere: 
Sphere(n=1.33, r=1.5000000013077628, center=[89.99999999855496, 59.99999999802843, 0.15803528497022953])


pixels: x = 179, y = 120, z = 0, units: x = 9.0e+01, y = 6.0e+01, z = 0.0e+00,


In [ ]:
#this is the block I've just been messing around in, the one above this is the one that actually works

path = r'C:\Users\kalis\Downloads'

p_size = 0.5 #pixel size
noise = 0.3 #average noise
med_n = 1.33 #medium refractive index
illum_wavelength = 0.633 #wavelength in vacuum
pol = (1,0) #polarization vector

#Define raw image
raw_holo = hp.load_image(path+r'\stuckparticles2.tif', spacing = p_size, channel = 0)
#hp.show(raw_holo)

#Background Correction
bkgd = hpci.load_average([path+r'\bkgd1.tif',path+r'\bkgd2.tif',path+r'\bkgd3.tif',
                          path+r'\bkgd4.tif'],refimg = raw_holo, channel = 0)
holo = bg_correct(raw_holo, bkgd)
#hp.show(holo)

#Update information stored with hologram and normalize
holo = hpc.update_metadata(holo, med_n, illum_wavelength, pol)
holo = normalize(holo)

#Project on 3D space
zstack = np.linspace(0,20,20) # defines space to project data onto
rec_vol = hp.propagate(holo,zstack) #propagates image onto zstack
hp.show(rec_vol)

#Define fit sphere
parameter_n = 1.33
parameter_r = 1.5
x = prior.Uniform(lower_bound=80, upper_bound=100, guess=91)
y = prior.Uniform(lower_bound=40, upper_bound=80, guess=60)
z = prior.Uniform(0, 20)
#parameter_center = (91, 60, 0.1)
parameter_sphere = Sphere(parameter_n, parameter_r, center=[x,y,z])
model=ExactModel(scatterer=parameter_sphere, calc_func=calc_holo)

#Visualize Estimate
#init_holo = calc_holo(rec_vol,parameter_sphere, med_n, illum_wavelength, pol, theory='auto')
#hp.show(init_holo)

#Perform Fit and display results
fit_result = hp.fit(holo, parameter_sphere)
print('Best Fit Sphere: ')
print(fit_result.scatterer)
print('\n')
hp.show(fit_result.hologram)